In [2]:
import gensim
import re
import numpy as np
from pyvi import ViTokenizer


In [3]:
with open("trainning.txt", encoding='UTF-8') as f:
    lines = f.read().split('\n')
for i in range(len(lines)):
    lines[i] = lines[i].split('\t')
X = []
Y = []
for line in lines:
    if(len(line) == 2):
        X.append(line[1])
        Y.append(line[0])


In [4]:
processed_lines = []
for line in X:
    line = gensim.utils.simple_preprocess(line)
    for i in range(len(line)):
        line[i].lower()
    line = ' '.join(line)
    line = ViTokenizer.tokenize(line)
    gensim.parsing.preprocessing.remove_stopwords(line)
    processed_lines.append(line)

In [5]:
for i in range(len(processed_lines)):
    processed_lines[i] = processed_lines[i].split(' ')
    processed_lines[i] = list(dict.fromkeys(processed_lines[i]))

In [6]:
wordList = []
filePath = 'wordListtesting.npy'
if os.path.isfile(filePath) is False:
    for line in processed_lines:
        for word in line:
            if word not in wordList:
                wordList.append(word)
    wordList = np.array(wordList)
    f = open(filePath, 'wb+')
    np.save(filePath,wordList)
    f.close()
else:
    f = open(filePath)
    wordList = np.load(filePath)



In [7]:
print(len(wordList))

40550


In [8]:
filePath = 'X_words.npy'
if os.path.isfile(filePath) is False:
    for i in range(len(X)):
        X[i].lower()
        X[i] = gensim.utils.simple_preprocess(X[i])
        X[i] = ' '.join(X[i])
        X[i] = ViTokenizer.tokenize(X[i])
        gensim.parsing.preprocessing.remove_stopwords(X[i])
        X[i] = X[i].split(' ')
    X = np.array(X)
    f = open(filePath, 'wb+')
    np.save(filePath,X)
    f.close()
else:
    f = open(filePath)
    X = np.load(filePath, allow_pickle = True)



In [9]:

print(X[0])

[&#39;gần&#39;, &#39;học_sinh&#39;, &#39;phải&#39;, &#39;học&#39;, &#39;trong&#39;, &#39;điều_kiện&#39;, &#39;tạm_bợ&#39;, &#39;nlđ&#39;, &#39;gần&#39;, &#39;học_sinh&#39;, &#39;trường&#39;, &#39;tiểu_học&#39;, &#39;đống&#39;, &#39;đa&#39;, &#39;quận&#39;, &#39;bình&#39;, &#39;thạnh&#39;, &#39;tphcm&#39;, &#39;đến&#39;, &#39;nay&#39;, &#39;vẫn&#39;, &#39;phải&#39;, &#39;được&#39;, &#39;gởi&#39;, &#39;học&#39;, &#39;tạm&#39;, &#39;tại&#39;, &#39;nơi&#39;, &#39;cách&#39;, &#39;nhà&#39;, &#39;khoảng&#39;, &#39;km&#39;, &#39;do&#39;, &#39;ngôi&#39;, &#39;trường&#39;, &#39;này&#39;, &#39;đã&#39;, &#39;xuống_cấp&#39;, &#39;trầm_trọng&#39;, &#39;theo&#39;, &#39;kế_hoạch&#39;, &#39;đầu&#39;, &#39;năm_học&#39;, &#39;trường&#39;, &#39;tiểu_học&#39;, &#39;đống&#39;, &#39;đa&#39;, &#39;được&#39;, &#39;khởi_công&#39;, &#39;xây_dựng&#39;, &#39;để&#39;, &#39;sang&#39;, &#39;năm&#39;, &#39;các&#39;, &#39;em&#39;, &#39;có&#39;, &#39;chỗ&#39;, &#39;học&#39;, &#39;nhưng&#39;, &#39;điều&#39;, &#39;đáng&#3

In [10]:
def toNumVector(X):
    NumVectorFull = []
    filePath = 'trainVector.npy'
    if os.path.isfile(filePath) is False:
        for word_array in X:
            NumVector = []
            for word in wordList:
                NumVector.append(word_array.count(word))
            NumVectorFull.append(NumVector)
        NumVectorFull = np.array(NumVectorFull)
        f = open(filePath, 'wb+')
        np.save(filePath,NumVectorFull)
        f.close()
        return NumVectorFull
    else:
        f = open(filePath)
        NumVectorFull = np.load(filePath, allow_pickle = True)
        f.close() 
        return NumVectorFull

In [11]:

NumVectorFull = toNumVector(X)

In [12]:
print(len(NumVectorFull))

5000


In [13]:
def transLabel(y):
    if y == '__CTXH__':
        return 0
    if y == '__SK__':
        return 1
    if y == '__TG__':
        return 2
    if y == '__DS__':
        return 3
    if y == '__TT__':
        return 4
    if y == '__KH__':
        return 5
    if y == '__PL__':
        return 6
    if y == '__VT__':
        return 7
    if y == '__KD__':
        return 8
    if y == '__VH__':
        return 9
    if y == 0:
        return '__CTXH__'
    if y == 1:
        return '__SK__'
    if y == 2:
        return '__TG__'
    if y == 3:
        return '__DS__'
    if y == 4:
        return '__TT__'
    if y == 5:
        return '__KH__'
    if y == 6:
        return '__PL__'
    if y == 7:
        return '__VT__'
    if y == 8:
        return '__KD__'
    if y == 9:
        return '__VH__'

In [14]:
for i in range(len(Y)):
    Y[i] = transLabel(Y[i])


In [15]:
print(len(Y))

5000


In [16]:
# đã xong phần feature selection, ta có tập train được chia thành 2 phần, tập X là vector đếm số lần xuất hiện của mỗi từ trong wordList trong mỗi văn bản, tập Y là label của mỗi văn bản

import pickle
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report

X_train, X_test, Y_train, Y_test =  train_test_split(NumVectorFull, Y,test_size=0.2,random_state=0)



In [17]:
print(X_test)

[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 4 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [18]:
def train(model, filePath, X_train, Y_train ):
    if os.path.isfile(filePath) is False:
        modelTrain = model
        modelTrain.fit(X_train, Y_train) 
        Y_predict = modelTrain.predict(X_test)
        print(classification_report(Y_test,Y_predict))
        pickle.dump(modelTrain, open(filePath, 'wb+'))
    else:
        model_load = pickle.load(open(filePath, 'rb'))
        Y_predict = model_load.predict(X_test)
        print(classification_report(Y_test,Y_predict))



In [19]:
#random forest
train(RandomForestClassifier(max_depth=10000, random_state=0), 'randomforest100.pkl', NumVectorFull, Y)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        99
           1       1.00      1.00      1.00       106
           2       1.00      1.00      1.00       102
           3       1.00      0.99      0.99        92
           4       1.00      1.00      1.00        82
           5       0.99      1.00      1.00       117
           6       1.00      1.00      1.00        89
           7       1.00      1.00      1.00       107
           8       1.00      1.00      1.00       105
           9       1.00      1.00      1.00       101

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [20]:
train(RandomForestClassifier(max_depth=10000, random_state=0), 'randomforest.pkl', X_train, Y_train)

              precision    recall  f1-score   support

           0       0.67      0.61      0.63        99
           1       0.84      0.92      0.88       106
           2       0.86      0.88      0.87       102
           3       0.81      0.80      0.81        92
           4       0.98      0.98      0.98        82
           5       0.89      0.75      0.81       117
           6       0.80      0.91      0.85        89
           7       0.90      0.83      0.86       107
           8       0.81      0.90      0.85       105
           9       0.88      0.86      0.87       101

    accuracy                           0.84      1000
   macro avg       0.84      0.85      0.84      1000
weighted avg       0.84      0.84      0.84      1000



In [21]:
train(GaussianNB(), 'naivebayes.pkl', X_train, Y_train)

              precision    recall  f1-score   support

           0       0.48      0.65      0.55        99
           1       0.86      0.82      0.84       106
           2       0.80      0.87      0.84       102
           3       0.70      0.75      0.73        92
           4       0.99      0.95      0.97        82
           5       0.76      0.64      0.69       117
           6       0.82      0.79      0.80        89
           7       0.89      0.77      0.82       107
           8       0.76      0.68      0.71       105
           9       0.76      0.81      0.78       101

    accuracy                           0.77      1000
   macro avg       0.78      0.77      0.77      1000
weighted avg       0.78      0.77      0.77      1000



In [22]:
train(svm.SVC(), 'SVM.pkl', X_train, Y_train)

              precision    recall  f1-score   support

           0       0.50      0.75      0.60        99
           1       0.93      0.77      0.85       106
           2       0.91      0.87      0.89       102
           3       0.63      0.83      0.72        92
           4       0.99      0.89      0.94        82
           5       0.88      0.79      0.83       117
           6       0.87      0.85      0.86        89
           7       0.89      0.73      0.80       107
           8       0.87      0.80      0.83       105
           9       0.88      0.83      0.86       101

    accuracy                           0.81      1000
   macro avg       0.83      0.81      0.82      1000
weighted avg       0.84      0.81      0.82      1000



now we use 100% of train data to build model
